In [1]:
import pandas as pd

# Read TSV file into DataFrame
df = pd.read_csv('../data/smolvlm_analysis_1.csv')

# Display basic info about the DataFrame
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
display(df.head())

Shape: (37, 52)
Columns: ['prompt2', 'ARMS.', 'Asian.', 'BAR.', 'BOOKLESS.', 'BOSTON.', 'Blonde.', 'Boy.', 'B~.', 'Cat.', 'Dog.', 'FEMALE.', 'Female.', 'Females.', 'GIRL.', 'GUY.', 'Handicap.', 'Implied, IGNORE.', 'Light is red.', 'MALE.', 'MAN.', 'Male.', 'Man.', 'Masculine.', 'Men.', 'Military.', 'Mom.', 'NO  The person(s) in this image are not present.', 'NO Answer: Female.', 'NO The person(s) in this image are not specified.', 'NO.', 'Neither.', 'No one is pictured in the image.', 'No one is present in the image.', 'No one is present in this image.', 'No one.', 'No person(s) in this picture.', 'One.', 'P.', 'PERSONAL.', 'PERSONALITY.', 'PERSONNEL 5.', 'PERSONS.', 'SAILORS.', 'STANLEY.', 'STARSHIP.', 'TERMINAL NOT INGREDIENTS.', 'UNAUTHORIZED.', 'VIP.', 'WOMEN.', 'Wetsuit.', 'Zebra.']

First few rows:


,prompt2,ARMS.,Asian.,BAR.,BOOKLESS.,BOSTON.,Blonde.,Boy.,B~.,Cat.,...,PERSONS.,SAILORS.,STANLEY.,STARSHIP.,TERMINAL NOT INGREDIENTS.,UNAUTHORIZED.,VIP.,WOMEN.,Wetsuit.,Zebra.
0,0.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,31.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9/9/2012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
yes_no_labels = df.iloc[:, 0].astype(str).tolist()
male_female_labels = df.columns.astype(str).tolist()[1:]
# Print the first row values and column headers
print("\nFirst row values:")
print(yes_no_labels[:5])
print("\nColumn headers:")
print(male_female_labels[:5])


First row values:
['0.', '1.', '14.', '31.', '9/9/2012']

Column headers:
['ARMS.', 'Asian.', 'BAR.', 'BOOKLESS.', 'BOSTON.']


In [3]:
# Read all labels
with open('../data/labels/yes_labels.txt', 'r') as file:
    yes_labels = [line.strip() for line in file.readlines()]

with open('../data/labels/no_labels.txt', 'r') as file:
    no_labels = [line.strip() for line in file.readlines()]

with open('../data/labels/other_labels_yn.txt', 'r') as file:
    other_labels_yn = [line.strip() for line in file.readlines()]

with open('../data/labels/male_labels.txt', 'r') as file:
    male_labels = [line.strip() for line in file.readlines()]

with open('../data/labels/female_labels.txt', 'r') as file:
    female_labels = [line.strip() for line in file.readlines()]

with open('../data/labels/other_labels_mf.txt', 'r') as file:
    other_labels_mf = [line.strip() for line in file.readlines()]

In [7]:
for label in yes_no_labels: 
    if not(label in yes_labels or label in no_labels or label in other_labels_yn):
        print(f"**ALERT** Label '{label}' not found in yes_labels.txt, no_labels.txt, or other_labels_yn.txt")
for label in male_female_labels: 
    if not(label in male_labels or label in female_labels or label in other_labels_mf):
        print(f"**ALERT** Label '{label}' not found in yes_labels.txt, no_labels.txt, or other_labels_yn.txt")

In [8]:
# Step 1: Replace values in the first column based on label lists
df.iloc[:, 0] = df.iloc[:, 0].apply(
    lambda x: 'yes' if x in yes_labels else 'no' if x in no_labels else 'other' if x in other_labels_yn else x
)

# Step 2: Group by that first column and aggregate (default is count per column)
aggregated_df_yn = df.groupby(df.columns[0]).sum().reset_index()

In [9]:
aggregated_df_yn.head()

,prompt2,ARMS.,Asian.,BAR.,BOOKLESS.,BOSTON.,Blonde.,Boy.,B~.,Cat.,...,PERSONS.,SAILORS.,STANLEY.,STARSHIP.,TERMINAL NOT INGREDIENTS.,UNAUTHORIZED.,VIP.,WOMEN.,Wetsuit.,Zebra.
0,no,1,0,1,0,2,1,2,1,1,...,1,1,1,1,1,2,0,17,0,5
1,other,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,yes,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,5,1,0


In [10]:
# Step 1: Map old column names to new ones
new_columns = [
    'male' if col in male_labels else
    'female' if col in female_labels else
    'other' if col in other_labels_mf else col
    for col in aggregated_df_yn.columns
]

# Step 2: Assign new column names
aggregated_df_yn.columns = new_columns

# Step 3: Group and aggregate columns with the same new name
final_agg_df = aggregated_df_yn.groupby(axis=1, level=0).sum()

final_agg_df.head()

/var/folders/3h/fqd_k18s5_sdlctlq7y54c2r0000gn/T/ipykernel_38124/4251662479.py:13: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  final_agg_df = aggregated_df_yn.groupby(axis=1, level=0).sum()


,female,male,other,prompt2
0,2868,635,99,no
1,9,15,1,other
2,615,745,13,yes
